In [38]:
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import FunctionTransformer, KBinsDiscretizer, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

In [39]:
import importlib
import helpers

importlib.reload(helpers)
from helpers import select_bmi_column, select_age_column, transform_bmi_features, transform_age_features

Chargement des données depuis un fichier CSV :

In [40]:
data_path = "Dataset_Brief.csv"
df = pd.read_csv(data_path)

Calcul du taux de valeurs manquantes :

In [41]:
missing_rate = df.isna().sum()/df.shape[0]
missing_rate

age         0.0
sex         0.0
bmi         0.0
children    0.0
smoker      0.0
region      0.0
charges     0.0
dtype: float64

Suppression des lignes contenant des valeurs manquantes :

In [42]:
df = df.dropna()

Identification et suppression des doublons :

In [43]:
duplicates = df.duplicated()
duplicates_data = df[duplicates]
df = df.drop_duplicates(subset=["age", "sex", "bmi", "children", "smoker", "region"], keep='first')

Sélection des caractéristiques (features) :

In [44]:
selected_features = ['age','sex' ,'bmi', 'children', 'smoker', 'region']
X = df[selected_features]
y = df['charges']

Division des données en ensembles d'entraînement et de test :

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

Identification des colonnes numériques et catégorielles :

In [46]:
num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

Création des pipelines de prétraitement :

In [47]:
onehotscale_pipeline = make_pipeline(OneHotEncoder(), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

ColumnTransformer (preprocessing) :

In [48]:
preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col),
        ('bmi_only', FunctionTransformer(select_bmi_column), ['bmi']),
        ('transform_bmi_features', FunctionTransformer(transform_bmi_features), ['bmi']),
        ('age_only', FunctionTransformer(select_age_column), ['age']),
        ('transform_age_features', FunctionTransformer(transform_age_features), ['age'])
    ]
)

Définition de la grille de recherche d'hyperparamètres :

In [49]:
param_grid = {'lasso__alpha': list(range(10, 55))}

Création d'un pipeline pour les caractéristiques polynomiales :

In [50]:
polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

Création du pipeline final :

In [51]:
my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)

Entraînement du pipeline final sur les données d'entraînement :

In [52]:
my_final_pipeline.fit(X_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder()),
                                                                  ('robustscaler',
                                                                   RobustScaler(with_centering=False))]),
                                                  ['sex', 'smoker', 'region']),
                                                 ('numerical',
                                                  Pipeline(steps=[('robustscaler',
                                                                   RobustScaler(with_centering=False))]),
                                                  ['age', 'bmi', 'children']),
                                                 ('bmi_only',
                                                  FunctionTransf...
                                                  FunctionTransformer(func=<function transform_bmi_features at 0x7f7202b78160>),
                                                  ['bmi']),
                                                 ('age_only',
                                                  FunctionTransformer(func=<function select_age_column at 0x7f7202b78dc0>),
                                                  ['age']),
                                                 ('transform_age_features',
                                                  FunctionTransformer(func=<function transform_age_features at 0x7f7202b79990>),
                                                  ['age'])])),
                ('pipeline',
                 Pipeline(steps=[('polynomialfeatures',
                                  PolynomialFeatures())]))])

Création du modèle de régression Lasso avec GridSearchCV :

In [53]:
model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

Recherche de grille pour trouver le meilleur hyperparamètre :

In [54]:
grid_search = GridSearchCV(model, param_grid,cv =5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('columntransformer',
                                                         ColumnTransformer(transformers=[('categorical',
                                                                                          Pipeline(steps=[('onehotencoder',
                                                                                                           OneHotEncoder()),
                                                                                                          ('robustscaler',
                                                                                                           RobustScaler(with_centering=False))]),
                                                                                          ['sex',
                                                                                           'smoker',
                                                                                           'region']),
                                                                                         ('numerical',
                                                                                          Pipeline(steps=[('robustscaler',
                                                                                                           RobustScaler(with_centering=False)...
                                                                                         ('transform_age_features',
                                                                                          FunctionTransformer(func=<function transform_age_features at 0x7f7202b79990>),
                                                                                          ['age'])])),
                                                        ('pipeline',
                                                         Pipeline(steps=[('polynomialfeatures',
                                                                          PolynomialFeatures())]))])),
                                       ('lasso',
                                        Lasso(max_iter=100000,
                                              random_state=42))]),
             param_grid={'lasso__alpha': [10, 11, 12, 13, 14, 15, 16, 17, 18,
                                          19, 20, 21, 22, 23, 24, 25, 26, 27,
                                          28, 29, 30, 31, 32, 33, 34, 35, 36,
                                          37, 38, 39, ...]})

Évaluation du modèle sur les données d'entraînement :

In [55]:
grid_score = grid_search.score(X_train, y_train)
best_alpha = grid_search.best_params_['lasso__alpha']

Prédiction sur les données de test :

In [56]:
y_pred = grid_search.predict(X_test)

Évaluation de la performance du modèle sur les données de test :

In [57]:
r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle après suppression des valeurs extrêmes:")
print(grid_search.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

Performance du modèle après suppression des valeurs extrêmes:
{'lasso__alpha': 36}
R2 Score: 0.8713
MAE: 2398.21
RMSE: 4326.94


Sauvegarde du modèle :

In [58]:
with open('modele.pkl', 'wb') as file:
    pickle.dump(grid_search, file)